In [1]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq ensemble-boxes
!pip install -qq mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!git clone https://github.com/alvaromoure/mmdetection.git
%cd mmdetection
!pip install -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 24866, done.
remote: Total 24866 (delta 0), reused 0 (delta 0), pack-reused 24866
Receiving objects: 100% (24866/24866), 37.84 MiB | 28.49 MiB/s, done.
Resolving deltas: 100% (17450/17450), done.
/kaggle/working/mmdetection
Obtaining file:///kaggle/working/mmdetection
  Preparing metadata (setup.py) ... - \ done
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl size=373746 sha256=6362ad5015c0fc3cbb860569c3fb6bd932a1d9f5285414f7871e9bcb8048d81c
  Stored in directory: /root/.cache/pip/wheels/06/f6/f9/9cc49c6de8e3cf27dfddd91bf46595a057141d4583a2adaf03
Successfully built pycocotools
  Running setup.py develop for mmdet


In [2]:
!pip install -q --upgrade wandb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires wandb<0.13.0,>=0.10.0, but you have wandb 0.13.5 which is incompatible.


# Imports

In [3]:
import wandb
import os
import shutil
import sys
import json
import torch,torchvision
from collections import defaultdict
from datetime import datetime
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector


import random
import numpy as np
from pathlib import Path

In [4]:
seed = 42
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb-key") 
wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Constants

In [6]:
FOLD = 4

In [7]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = 'TFM - Detection Model Testing'
EXP_NAME = f'cascade_rcnn_x101_Fold{FOLD}'


In [8]:
model_name = f'cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco'

# Wandb Initiation and file downloading

In [9]:
import pytz
timezone = pytz.timezone('Europe/Madrid')
now = datetime.now(timezone)

dt_string = now.strftime("%d/%m_%H:%M")

In [10]:
run = wandb.init(project=PROJECT_NAME,
                 name=f'Downloading_files_{EXP_NAME}_{dt_string}',
                job_type='file_transfer',
                notes='')


wandb: Currently logged in as: alvaromoureupm. Use `wandb login --relogin` to force relogin


In [11]:
artifact = run.use_artifact('alvaromoureupm/Final-Covid19-Detection/5_fold_train_val_coco_files:v2')
artifact_dir = artifact.download()
run.finish()

wandb:   10 of 10 files downloaded.  


In [12]:
for file in os.listdir(artifact_dir):
    shutil.copyfile(os.path.join(artifact_dir,file),f'/kaggle/working/{file}')

In [13]:
train_ann_file = f'/kaggle/working/coco_train_512x512_fold_{FOLD}.json'
val_ann_file = f'/kaggle/working/coco_val_512x512_fold_{FOLD}.json'

# Model Training

In [14]:
EPOCHS = 15

In [15]:
baseline_cfg_path = f'/kaggle/working/mmdetection/configs/{model_name}.py'
cfg = Config.fromfile(baseline_cfg_path)
job_folder = f'/kaggle/working/cascade_rcnn'
if not os.path.exists(job_folder):
    os.makedirs(job_folder)
cfg.work_dir = job_folder

In [16]:
cfg.model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d'),
        groups=32,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(
            type='SmoothL1Loss', beta=0.1111111111111111, loss_weight=1.0)),
    roi_head=dict(
        type='CascadeRoIHead',
        num_stages=3,
        stage_loss_weights=[1, 0.5, 0.25],
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0.0, 0.0, 0.0, 0.0],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0.0, 0.0, 0.0, 0.0],
                    target_stds=[0.05, 0.05, 0.1, 0.1]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0.0, 0.0, 0.0, 0.0],
                    target_stds=[0.033, 0.033, 0.067, 0.067]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))
        ]),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=0,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=[
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.5,
                    neg_iou_thr=0.5,
                    min_pos_iou=0.5,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                pos_weight=-1,
                debug=False),
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.6,
                    neg_iou_thr=0.6,
                    min_pos_iou=0.6,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                pos_weight=-1,
                debug=False),
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.7,
                    neg_iou_thr=0.7,
                    min_pos_iou=0.7,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                pos_weight=-1,
                debug=False)
        ]),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100)))

In [17]:
albu_train_transforms = [
    #dict(type='RandomResizedCrop',height=image_scale[0],width=image_scale[1],scale=(0.25,1.0),ratio=(0.75,1.3333),interpolation=1,p=1.0),
    dict(type='Rotate',limit=10,interpolation=1,border_mode=0,value=0,p=0.25),
    dict(type='HorizontalFlip',p=0.5),
    dict(
        type="OneOf",
        transforms=[
            dict(type="MotionBlur", p=0.2),
            dict(type="MedianBlur", p=0.1, blur_limit=3),
            dict(type='Blur',p=0.1,blur_limit=3)
        ],
        p=0.25),
    #dict(type='ShiftScaleRotate', shift_limit=0.0625,scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.2)
]

In [18]:
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333,800), keep_ratio=True),
    dict(type='EqualizeTransform', mode='cv', p=1,always_apply=True),
    dict(
        type='Albu',
        transforms=[dict(type='CLAHE', p=1),
                    *albu_train_transforms],
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

In [19]:
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Equalize', mode='cv', p=1, always_apply=True),
            dict(type='CLAHE', p=1),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [20]:
cfg.data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type='CocoDataset',
        ann_file='/kaggle/working/coco_train_512x512_fold_0.json',
        img_prefix=
        '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
        ],
        classes=('Covid_Opacity', )),
    val=dict(
        type='CocoDataset',
        ann_file='/kaggle/working/coco_val_512x512_fold_0.json',
        img_prefix=
        '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', )),
    test=dict(
        type='CocoDataset',
        ann_file='/kaggle/working/coco_val_512x512_fold_0.json',
        img_prefix=
        '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/test_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', )),
    samplers_per_gpu=32)

In [21]:
cfg.evaluation = dict(interval=1, metric='bbox')
cfg.optimizer = dict(type='SGD', lr=0.001, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(
    policy='CosineAnnealing',
    by_epoch=False,
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.003,
    min_lr=1e-07)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=10)
cfg.checkpoint_config = dict(interval=1)

cfg.custom_hooks = [dict(type='NumClassCheckHook')]
cfg.dist_params = dict(backend='nccl')
cfg.log_level = 'INFO'
cfg.load_from = None
cfg.resume_from = None
cfg.workflow = [('train', 1)]
cfg.opencv_num_threads = 0
cfg.mp_start_method = 'fork'
cfg.auto_scale_lr = dict(enable=False, base_batch_size=16)
cfg.work_dir = '/kaggle/working/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco'
cfg.seed = 42
cfg.gpu_ids = [0]
cfg.total_epochs = 10
cfg.classes = ('Covid_Opacity', )
cfg.evaluation_metric = 'bbox'
cfg.evaluation_interval = 1

In [22]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project=PROJECT_NAME,
                                              name=f'{EXP_NAME}_{dt_string}',
                                              job_type='model_train',
                                              entity='alvaromoureupm',
                                             notes=f'Notebook: {NOTEBOOK_NAME}',
                                             config=cfg._cfg_dict.to_dict()),
                             log_artifact=True,
                             interval = 100
                            )
                       ]

In [23]:
cfg.dump('/kaggle/working/cascade_rcnn/cascade_rcnn.py')

In [24]:
!python tools/train.py /kaggle/working/cascade_rcnn/cascade_rcnn.py

/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2022-11-09 11:52:34,227 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NV

# Loging the best model in Wandb

log_file = f'{job_folder}/None.log.json'

# Source: mmdetection/tools/analysis_tools/analyze_logs.py 
def load_json_logs(json_logs):
    log_dicts = [dict() for _ in json_logs]
    for json_log, log_dict in zip(json_logs, log_dicts):
        with open(json_log, 'r') as log_file:
            for line in log_file:
                log = json.loads(line.strip())
                # skip lines without `epoch` field
                if 'epoch' not in log:
                    continue
                epoch = log.pop('epoch')
                if epoch not in log_dict:
                    log_dict[epoch] = defaultdict(list)
                for k, v in log.items():
                    log_dict[epoch][k].append(v)
    return log_dicts

log_dict = load_json_logs([log_file])
best_epoch = np.argmax([item['bbox_mAP'][0] for item in log_dict[0].values()])+1
best_epoch

run = wandb.init(project=PROJECT_NAME,
                 name=f'Saving_model{EXP_NAME}_{dt_string}',
                job_type='model_save',
                notes=f'Notebook: {NOTEBOOK_NAME}')

artifact = wandb.Artifact(EXP_NAME, type='model')

artifact.add_file(f'{job_folder}/epoch_{best_epoch}.pth')
artifact.add_file(cfg_path)

run.log_artifact(artifact)
run.finish()